In [20]:
import csv
import math

import matplotlib.pyplot as plt
import numpy as np

from keras import applications
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input


In [21]:
# read the CSV into memory
prices = []
image_paths = []

data_path = "../datasets/bikes_im/"
with open("../datasets/bikes_filtered.csv") as file:
    reader = csv.reader(file)
    i = -1
    for row in reader:
        i += 1
        index = row[0]
        name = row[1]
        msrp = row[2]
        
        image_path = data_path + index + '.jpg'
        image_paths.append(image_path)
        prices.append(int(msrp))

train_indices = np.load("bikes_train_indices.npy")
test_indices = np.load("bikes_test_indices.npy")
print(train_indices.shape)
print(test_indices.shape)

(19658,)
(2185,)


## Sliding Windows

Image defaulted to image_paths[0]. To pick the right image, change the index you are using.
Runs through window sizes of 28x28, so total of 64 iterations for a 224x224 image.
All sliding window images saved to n

In [22]:
# Take a data point and run it through the neural network
# Return the predicted value and calculate the MSE
def evaluate(model, index):
    msrp = prices[index]
    path = image_paths[index]
    img = image.load_img(path, target_size=(224, 224))
    data = np.expand_dims(image.img_to_array(img), axis=0)
    # TODO: the input should be preprocessed before feeding it in
    prediction = model.predict(data, msrp)
    
    #print("Bike index: " + str(index))
    #print("Actual price: " + str(msrp))
    #print("Predicted price: " + str(prediction))
    mse = (prediction-msrp)**2
    return mse


img = image.load_img(image_paths[0], target_size=(224, 224))
pixels = img.load() # create the pixel map
for x in range(0, 30):
    for y in range(0, 30):
        pixels[x,y] = (117, 117, 117) # set the colour accordingly
img.save("sliding_window1.png")

sliding_window_images = np.zeros((64, 224, 224, 3))
for i in range(int(224/28)):    # for every col:
    for j in range(int(224/28)):    # For every row
        #pixels = orig_img.load() # create the pixel map
        #pixels[i,j] = (117, 117, 117) # set the colour accordingly
        #img.show()
        start_i = i * 28
        end_i = i*28 + 28
        start_j = j * 28
        end_j = j*28 + 28
        print(start_i, end_i, start_j, end_j)
        
        orig_img = image.load_img(image_paths[0], target_size=(224, 224))
        pixels = orig_img.load() # create the pixel map
        for y in range(start_i, end_i):
            for x in range(start_j, end_j):
                
                pixels[x,y] = (117, 117, 117) # set the colour accordingly
                
        sliding_i = i*8 + j
        sliding_window_images[sliding_i, :, :, :] = image.img_to_array(orig_img)
        #image.img_to_array(img)
        #orig_img.save('sliding_windows/sliding_window'+str(i)+str(j)+'.png')
#print(num_window_images)
         

0 28 0 28
0 28 28 56
0 28 56 84
0 28 84 112
0 28 112 140
0 28 140 168
0 28 168 196
0 28 196 224
28 56 0 28
28 56 28 56
28 56 56 84
28 56 84 112
28 56 112 140
28 56 140 168
28 56 168 196
28 56 196 224
56 84 0 28
56 84 28 56
56 84 56 84
56 84 84 112
56 84 112 140
56 84 140 168
56 84 168 196
56 84 196 224
84 112 0 28
84 112 28 56
84 112 56 84
84 112 84 112
84 112 112 140
84 112 140 168
84 112 168 196
84 112 196 224
112 140 0 28
112 140 28 56
112 140 56 84
112 140 84 112
112 140 112 140
112 140 140 168
112 140 168 196
112 140 196 224
140 168 0 28
140 168 28 56
140 168 56 84
140 168 84 112
140 168 112 140
140 168 140 168
140 168 168 196
140 168 196 224
168 196 0 28
168 196 28 56
168 196 56 84
168 196 84 112
168 196 112 140
168 196 140 168
168 196 168 196
168 196 196 224
196 224 0 28
196 224 28 56
196 224 56 84
196 224 84 112
196 224 112 140
196 224 140 168
196 224 168 196
196 224 196 224
